In [9]:
import config
import telebot
import os, time, random
from SQLighter import SQLighter
import utils

In [10]:
bot = telebot.TeleBot(config.token)

@bot.message_handler(commands=['test'])
def find_file_ids(message):
    for file in os.listdir('music/'):
        if file.split('.')[-1] == 'ogg':
            f = open('music/'+file, 'rb')
            msg = bot.send_voice(message.chat.id, f, None)
            # А теперь отправим вслед за файлом его file_id
            bot.send_message(message.chat.id, msg.voice.file_id, reply_to_message_id=msg.message_id)
        time.sleep(3)

@bot.message_handler(commands=["start"])
def say_hello(message): # Название функции не играет никакой роли
    bot.send_message(message.chat.id, 'Привет, я бот МЦ. Какая информация Вас интересует?')    
    


# @bot.message_handler(content_types=["text"])
# def repeat_all_messages(message): # Название функции не играет никакой роли
#     bot.send_message(message.chat.id, message.text)
    
@bot.message_handler(commands=['game'])
def game(message):
    # Подключаемся к БД
    db_worker = SQLighter(config.database_name)
    
    utils.count_rows() # вроде как нужно чтобы сохранить в хранилище музыку из базы 
    
    
    
    # Получаем случайную строку из БД
    row = db_worker.select_single(random.randint(1, utils.get_rows_count()))
    # Формируем разметку
    markup = utils.generate_markup(row[2], row[3])
    # Отправляем аудиофайл с вариантами ответа
    bot.send_voice(message.chat.id, row[1], reply_markup=markup)
    # Включаем "игровой режим"
    utils.set_user_game(message.chat.id, row[2])
    # Отсоединяемся от БД
    db_worker.close()    
    
@bot.message_handler(func=lambda message: True, content_types=['text'])
def check_answer(message):
    # Если функция возвращает None -> Человек не в игре
    answer = utils.get_answer_for_user(message.chat.id)
    # Как Вы помните, answer может быть либо текст, либо None
    # Если None:
    if not answer:
        bot.send_message(message.chat.id, 'Чтобы начать игру, выберите команду /game')
    else:
        # Уберем клавиатуру с вариантами ответа.
        keyboard_hider = types.ReplyKeyboardRemove()
        # Если ответ правильный/неправильный
        if message.text == answer:
            bot.send_message(message.chat.id, 'Верно!', reply_markup=keyboard_hider)
        else:
            bot.send_message(message.chat.id, 'Увы, Вы не угадали. Попробуйте ещё раз!', reply_markup=keyboard_hider)
        # Удаляем юзера из хранилища (игра закончена)
        utils.finish_user_game(message.chat.id)
  
    
if __name__ == '__main__':
    bot.infinity_polling()    

2020-11-13 15:47:03,456 (util.py:75 WorkerThread10) ERROR - TeleBot: "IndexError occurred, args=('list index out of range',)
Traceback (most recent call last):
  File "C:\Users\AmanovRA\Anaconda3\lib\site-packages\telebot\util.py", line 69, in run
    task(*args, **kwargs)
  File "<ipython-input-10-7cf2024f85a3>", line 35, in game
    markup = utils.generate_markup(row[2], row[3])
IndexError: list index out of range
"


KeyboardInterrupt: 